# Data Wrangling: Join, Combine, 

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Hierarchical Indexing

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [11]:
data['b']
data['b':'c']
data.loc[['b', 'd']]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

In [18]:
data.loc[:, 2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

In [27]:
data.unstack()

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,NaN,1.965781
c,1.393406,0.092908,NaN
d,NaN,0.281746,0.769023


In [28]:
data.unstack().stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

In [29]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [30]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [79]:
frame['Ohio']
frame.loc(axis=0)['a'].loc(axis=1)[:,'Green']
frame.loc['a',(slice(None),'Green')]
frame.loc['a',('Ohio','Green')]

key2
1    0
2    3
Name: (Ohio, Green), dtype: int64

MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

### Reordering and Sorting Levels

In [84]:
frame.swaplevel('key1', 'key2')
frame.swaplevel('state', 'color', axis=1)['Green']

state      Ohio  Colorado
key1 key2                
a    1        0         2
     2        3         5
b    1        6         8
     2        9        11

In [96]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [94]:
f2 = frame.sort_index(level=1)
f2
f3 = frame.swaplevel(0,1)
f3
f4 = frame.swaplevel(0, 1).sort_index(level=0)
f4

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

In [106]:
frame.sum(level='key2')
frame.sum(level='color', axis=1)
print(frame['Ohio':].sum(level='color', axis=1))

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10


### Indexing with a DataFrame's columns

In [107]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [108]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [109]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [124]:
frame2.reset_index(inplace=True)

In [125]:
frame2


,alan,c,d,a,b
0,0,one,0,0,7
1,1,one,1,1,6
2,2,one,2,2,5
3,3,two,0,3,4
4,4,two,1,4,3
5,5,two,2,5,2
6,6,two,3,6,1


## Combining and Merging Datasets

### Database-Style DataFrame Joins

In [132]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
print(df1)
print('+'*20)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
++++++++++++++++++++
  key  data2
0   a      0
1   b      1
2   d      2


In [128]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [129]:
pd.merge(df2,df1)

,key,data2,data1
0,a,0,2
1,a,0,4
2,a,0,5
3,b,1,0
4,b,1,1
5,b,1,6


In [130]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [ ]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

In [133]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [134]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df1
df2
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


In [135]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [136]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [138]:
pd.merge(left, right, on='key1')
# pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

In [141]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'c', 'b', 'a'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
left1
right1
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
5,a,5,3.5
1,b,1,7.0
4,b,4,7.0


In [142]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
5,a,5,3.5
1,b,1,7.0
4,b,4,7.0
3,c,3,NaN


In [145]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)}, index=['a','b','c','d','e'])
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
print(lefth)
print('+'*20)
print(righth)

     key1  key2  data
a    Ohio  2000   0.0
b    Ohio  2001   1.0
c    Ohio  2002   2.0
d  Nevada  2001   3.0
e  Nevada  2002   4.0
++++++++++++++++++++
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [146]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

,key1,key2,data,event1,event2
a,Ohio,2000,0.0,4.0,5.0
a,Ohio,2000,0.0,6.0,7.0
b,Ohio,2001,1.0,8.0,9.0
c,Ohio,2002,2.0,10.0,11.0
d,Nevada,2001,3.0,0.0,1.0
e,Nevada,2002,4.0,NaN,NaN
e,Nevada,2000,NaN,2.0,3.0


In [147]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
left2
right2
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [148]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [149]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,c,3,NaN
4,b,4,7.0
5,a,5,3.5


In [ ]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another
left2.join([right2, another])
left2.join([right2, another], how='outer')

### Concatenating Along an Axis

In [154]:
arr = np.arange(12).reshape((3, 4))
arr
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [160]:
arr2 = np.arange(12,15).reshape(3,1)
arr2

array([[12],
       [13],
       [14]])

In [168]:
np.concatenate([arr,arr2], axis=1)

array([[ 0,  1,  2,  3, 12],
       [ 4,  5,  6,  7, 13],
       [ 8,  9, 10, 11, 14]])

In [208]:
arr3 = np.arange(27).reshape(3,3,3)
arr3
arr4 = np.arange(27,54).reshape(3,3,3)
np.concatenate([arr3,arr4],axis=2)[0][1][3]


30

In [155]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
a
s1 , s2, s3

(a    0
 b    1
 dtype: int64,
 c    2
 d    3
 e    4
 dtype: int64,
 f    5
 g    6
 dtype: int64)

In [205]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [206]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [212]:
s4 = pd.concat([s1, s3])
s4
pd.concat([s1, s4], axis=1)
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


In [215]:
# join_axes deprecated as of pandas .025
# pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])
pd.concat([s1, s4], axis=1).reindex(['a', 'c', 'b', 'e'])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [216]:
pd.concat([s1, s1,s3])

a    0
b    1
a    0
b    1
f    5
g    6
dtype: int64

In [217]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In [218]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [221]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
df1
df2
print(df1, '\n', df2)
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

   one  two
a    0    1
b    2    3
c    4    5 
    three  four
a      5     6
c      7     8


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [222]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [223]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [224]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1
df2

,b,d,a
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614


In [244]:
pd.concat([df1, df2], ignore_index=True)
pd.concat([df1, df1], axis=1, keys=['one','two'],          names=['upper', 'lower'])

upper  one           two         
lower    a    b   c    a    b   c
0      1.0  NaN   2  1.0  NaN   2
1      NaN  2.0   6  NaN  2.0   6
2      5.0  NaN  10  5.0  NaN  10
3      NaN  6.0  14  NaN  6.0  14

### Combining Data with Overlap

In [226]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
a
b
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [234]:
a
b
pd.isnull(a)
b[:-2] , a[2:]

(f    0.0
 e    1.0
 d    2.0
 c    3.0
 dtype: float64,
 d    NaN
 c    3.5
 b    4.5
 a    NaN
 dtype: float64)

In [232]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [235]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df1
df2
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## Reshaping and Pivoting

### Reshaping with Hierarchical Indexing

In [245]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [246]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [248]:
result.index

MultiIndex([(    'Ohio',   'one'),
            (    'Ohio',   'two'),
            (    'Ohio', 'three'),
            ('Colorado',   'one'),
            ('Colorado',   'two'),
            ('Colorado', 'three')],
           names=['state', 'number'])

In [249]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [253]:
result.unstack(0)
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [254]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [258]:
data2.unstack()
data2.unstack().stack()
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [260]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [275]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
print(df.columns, df.index)
df
df.unstack('state')
df.unstack()


Index(['left', 'right'], dtype='object', name='side') MultiIndex([(    'Ohio',   'one'),
            (    'Ohio',   'two'),
            (    'Ohio', 'three'),
            ('Colorado',   'one'),
            ('Colorado',   'two'),
            ('Colorado', 'three')],
           names=['state', 'number'])


side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorado    3   4     5     8   9    10

In [272]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting “Long” to “Wide” Format

In [278]:
data = pd.read_csv('examples/macrodata.csv')
data.head()
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203, freq='Q-DEC')

In [301]:
t = pd.DataFrame(np.arange(9).reshape(3,3), index=['a','b', 'c'], columns=pd.Index(['x','y','z'], name='item'))
t= t.reindex(columns=pd.Index(['x','y','z'], name='item2'))

In [304]:
t.stack().index

MultiIndex([('a', 'x'),
            ('a', 'y'),
            ('a', 'z'),
            ('b', 'x'),
            ('b', 'y'),
            ('b', 'z'),
            ('c', 'x'),
            ('c', 'y'),
            ('c', 'z')],
           names=[None, 'item2'])

In [289]:
data.stack().reset_index()[:10]
#data.columns

,date,item,0
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [305]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [317]:
pivoted = ldata.pivot('date', 'item')
pivoted.columns

MultiIndex([( 'value',    'infl'),
            ( 'value', 'realgdp'),
            ( 'value',   'unemp'),
            ('value2',    'infl'),
            ('value2', 'realgdp'),
            ('value2',   'unemp')],
           names=[None, 'item'])

In [316]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.866544
1,1959-03-31 23:59:59.999999999,infl,0.000,1.079464
2,1959-03-31 23:59:59.999999999,unemp,5.800,-0.397475
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,1.487506
4,1959-06-30 23:59:59.999999999,infl,2.340,0.558011
5,1959-06-30 23:59:59.999999999,unemp,5.100,0.614350
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,0.427964
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.271959
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.738759
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-0.508070


In [312]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]
pivoted.columns
#pivoted['value'][:5]

MultiIndex([( 'value',    'infl'),
            ( 'value', 'realgdp'),
            ( 'value',   'unemp'),
            ('value2',    'infl'),
            ('value2', 'realgdp'),
            ('value2',   'unemp')],
           names=[None, 'item'])

In [318]:
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  1.079464  0.866544   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1  0.558011  1.487506   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.271959  0.427964   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  1.315273 -0.508070   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2  0.193262  0.573192   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2  0.575008  1.279619   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -1.237023 -0.765794   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.397475  
1959-06-30 23:59:59.999999999  0.614350  
1959-09-30 23:59:59.999999999  0.738759  
1959-12-31 23:59:59.999999999  0.445930  
1960-03-31 23:59:59.999999999  0.442924  
1960-06-30 23:59:59.999999999  0.433936  
1960-09-30 23:59:59.999999999 -0.536722

### Pivoting “Wide” to “Long” Format

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

In [ ]:
melted = pd.melt(df, ['key'])
melted

In [ ]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

In [ ]:
reshaped.reset_index()

In [ ]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

In [ ]:
pd.melt(df, value_vars=['A', 'B', 'C'])
pd.melt(df, value_vars=['key', 'A', 'B'])

## Conclusion